In [1]:
import os
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stop_words = stopwords.words('english')
stop_words += ['mr','would','say','lt', 'p', 'gt', 'amp', 'nbsp',
               'bill','speaker','us','going','act','gentleman',
              'gentlewoman','chairman','nay','yea','thank']
pathnames = ['./convote_v1.1/data_stage_one/'+wor+'/' for 
             wor in ['development_set','training_set']]
#pathnames = ['./data_stage_one/training_set/']
docs,docLen=[],0
for path in pathnames:
    for filename in os.listdir(path):
        with open(path+filename,'r') as inp:
            f=inp.read()
            words=word_tokenize(f)
            words = [w.lower() for w in words]
            noPunc = [w.translate(None,string.punctuation)
                      for w in words]
            noEmp = [w for w in noPunc if w.isalpha()]
            noStop = [w for w in noEmp if not w
                      in stop_words]
            stemmed = [porter.stem(w) for w in noStop]
            stemmed = [w for w in stemmed if not w
                      in stop_words]
        docLen+=len(stemmed)
        docs.append(stemmed)
        #docs.append(noStop)
l = len(docs)
print ("Total Number of documents = %d"%l)
print("Average words per document = %d"%(docLen/l))

Total Number of documents = 6362
Average words per document = 118


In [2]:
from gensim import corpora, models, similarities
import tempfile
TMP = tempfile.gettempdir()
dcy = corpora.Dictionary(docs)
print("Total vocabulary size = %d"%len(dcy))
dcy.save(os.path.join(TMP,'cong.dict'))
corpus = [dcy.doc2bow(text) for text in docs]
corpora.MmCorpus.serialize(os.path.join
                           (TMP,'congCorp.mm'),corpus)
#for c in corpus[:10]:
#    print(c)
tfidf = models.TfidfModel(corpus, normalize=True)
tfidf_corpus = tfidf[corpus]
tfidf_corpus = corpus
lda = models.LdaModel(tfidf_corpus, id2word=dcy, 
                      num_topics=10)
lda.print_topics(3,10)


Total vocabulary size = 15547


[(0,
  u'0.021*"iraq" + 0.016*"iraqi" + 0.009*"peopl" + 0.009*"year" + 0.009*"work" + 0.007*"forc" + 0.007*"resolut" + 0.007*"committe" + 0.006*"nation" + 0.006*"report"'),
 (5,
  u'0.016*"energi" + 0.012*"oil" + 0.010*"ocean" + 0.009*"amend" + 0.008*"ga" + 0.007*"price" + 0.007*"time" + 0.007*"need" + 0.006*"year" + 0.005*"want"'),
 (3,
  u'0.011*"fund" + 0.010*"budget" + 0.009*"year" + 0.008*"rule" + 0.008*"vote" + 0.007*"amend" + 0.007*"committe" + 0.006*"hous" + 0.006*"appropri" + 0.006*"resolut"')]

In [3]:
lda.print_topics(-1,20)

[(0,
  u'0.021*"iraq" + 0.016*"iraqi" + 0.009*"peopl" + 0.009*"year" + 0.009*"work" + 0.007*"forc" + 0.007*"resolut" + 0.007*"committe" + 0.006*"nation" + 0.006*"report" + 0.005*"unit" + 0.005*"recommend" + 0.005*"support" + 0.005*"secur" + 0.005*"commiss" + 0.005*"new" + 0.005*"presid" + 0.005*"issu" + 0.004*"time" + 0.004*"hous"'),
 (1,
  u'0.025*"tax" + 0.012*"plan" + 0.011*"cut" + 0.009*"benefit" + 0.009*"year" + 0.009*"billion" + 0.008*"deficit" + 0.008*"american" + 0.008*"republican" + 0.008*"pension" + 0.006*"worker" + 0.006*"peopl" + 0.006*"increas" + 0.005*"support" + 0.005*"famili" + 0.005*"today" + 0.005*"secur" + 0.005*"fund" + 0.005*"retir" + 0.005*"percent"'),
 (2,
  u'0.017*"cut" + 0.015*"tax" + 0.012*"peopl" + 0.011*"health" + 0.009*"fund" + 0.009*"american" + 0.009*"million" + 0.007*"care" + 0.007*"program" + 0.007*"need" + 0.007*"increas" + 0.006*"year" + 0.006*"support" + 0.006*"percent" + 0.006*"provid" + 0.006*"dividend" + 0.006*"plan" + 0.005*"work" + 0.005*"child